In [1]:
import gym 
import retro
import numpy as np
import  gym
from gym import Env


from torch.utils.tensorboard import SummaryWriter
import retro
import cv2
from gym.spaces import Box,MultiBinary
import numpy as np
import optuna
import tqdm as notebook_tqdm
from stable_baselines3.common.vec_env import DummyVecEnv, VecVideoRecorder ,VecFrameStack
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

c:\Users\rifat\miniconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from gym import Env
from gym.spaces import Box, MultiBinary
import numpy as np
import cv2
import retro

In [3]:
class KungFuKid(Env):
    def __init__(self) -> None:
        super().__init__()
        self.observation_space=Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)
        self.action_space=MultiBinary(9)
        self.game = retro.make(game='FantasyZoneTheMaze-Sms',use_restricted_actions=retro.Actions.FILTERED)

    def preprocess_observation(self,obs):
        gray = cv2.cvtColor(obs, cv2.COLOR_RGB2GRAY)
        resize = cv2.resize(gray, (84, 84),interpolation=cv2.INTER_AREA)
        chennels = np.expand_dims(resize, axis=2)
        return chennels
   

    def step(self, action):
        obs, reward, done, info = self.game.step(action)
        obs = self.preprocess_observation(obs)
        frame_delta = obs - self.previous_frame
        self.previous_frame = obs

        score_reward = (info['score'] - self.score) / 10

        # Lives punishment
        lives_punishment = 0
        if info['lives'] < self.lives:
            lives_punishment = 10

        reward = score_reward - lives_punishment

        self.score = info['score']
        self.lives = info['lives']

        return frame_delta, reward, done, info


    def reset(self):
        obs = self.game.reset()
        obs = self.preprocess_observation(obs)
        self.previous_frame = obs
        self.score = 0
        self.lives = 5
        return obs

    def render (self,*args,**kwargs):
        self.game.render()

    def close(self):
        self.game.close()


In [4]:
import os 
import os 
models_dir ='models/FantasyZoneTheMaze-Sms'
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

log_dir = 'logs/FantasyZoneTheMaze-Sms'

if not os.path.exists(log_dir):
    os.makedirs(log_dir)

opt_dir = 'opt/FantasyZoneTheMaze-Sms'

if not os.path.exists(opt_dir):
    os.makedirs(opt_dir)


In [5]:
def optimize_ppo(trial):
    return {
        'n_steps': trial.suggest_int('n_steps', 8192, 24576),
        'gamma': trial.suggest_loguniform('gamma', 0.9, 0.9999),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1),
        'ent_coef': trial.suggest_loguniform('ent_coef', 1e-5, 1e-3),
        'clip_range': trial.suggest_uniform('clip_range', 0.1, 0.4),
        'gae_lambda': trial.suggest_uniform('gae_lambda', 0.8, 0.99),
    }

In [6]:
# Run a training loop and return mean reward 
def optimize_agent(trial):
    try:
        model_params = optimize_ppo(trial) 

        # Create environment 
        env = KungFuKid()
        env = Monitor(env, log_dir)
        env = DummyVecEnv([lambda: env])
        env = VecFrameStack(env, 4, channels_order='last')

        # Create algo 
        model = PPO('CnnPolicy', env,batch_size=4096, tensorboard_log=log_dir, verbose=0, **model_params)
        # model.learn(total_timesteps=30000)
        model.learn(total_timesteps=100000)

        # Evaluate model 
        mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=5)
        env.close()

        SAVE_PATH = os.path.join(opt_dir, 'trial_{}_best_model'.format(trial.number))
        model.save(SAVE_PATH)

        return mean_reward

    except Exception as e:
        return -1000

In [7]:
study = optuna.create_study(direction='maximize')
study.optimize(optimize_agent, n_trials=5, n_jobs=1)

[I 2023-06-02 19:52:20,774] A new study created in memory with name: no-name-f9ce8945-cd58-4877-a783-2553be5b0b1e
C:\Users\rifat\AppData\Local\Temp\ipykernel_13664\1038144062.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'gamma': trial.suggest_loguniform('gamma', 0.9, 0.9999),
C:\Users\rifat\AppData\Local\Temp\ipykernel_13664\1038144062.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1),
C:\Users\rifat\AppData\Local\Temp\ipykernel_13664\1038144062.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://gi

In [8]:
study.best_params

{'n_steps': 13659,
 'gamma': 0.9156395542666173,
 'learning_rate': 0.00019516296544646962,
 'ent_coef': 0.0001034325135216129,
 'clip_range': 0.1396571735161551,
 'gae_lambda': 0.9041867823711087}

In [12]:
# import pickle

# # Save study.best_params to a file
# with open('best_params.pkl', 'wb') as file:
#     pickle.dump(study.best_params, file)


In [ ]:
# import pickle

# # Load study.best_params from a file
# with open('best_params.pkl', 'rb') as file:
#     best_params = pickle.load(file)


In [11]:
study.best_trials

[FrozenTrial(number=3, state=TrialState.COMPLETE, values=[-15.2], datetime_start=datetime.datetime(2023, 6, 2, 20, 15, 27, 934091), datetime_complete=datetime.datetime(2023, 6, 2, 20, 23, 5, 258650), params={'n_steps': 13659, 'gamma': 0.9156395542666173, 'learning_rate': 0.00019516296544646962, 'ent_coef': 0.0001034325135216129, 'clip_range': 0.1396571735161551, 'gae_lambda': 0.9041867823711087}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_steps': IntDistribution(high=24576, log=False, low=8192, step=1), 'gamma': FloatDistribution(high=0.9999, log=True, low=0.9, step=None), 'learning_rate': FloatDistribution(high=1.0, log=True, low=1e-05, step=None), 'ent_coef': FloatDistribution(high=0.001, log=True, low=1e-05, step=None), 'clip_range': FloatDistribution(high=0.4, log=False, low=0.1, step=None), 'gae_lambda': FloatDistribution(high=0.99, log=False, low=0.8, step=None)}, trial_id=3, value=None)]

In [13]:
# Import os for file path management
import os 
# Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [14]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [15]:
models_dir

'models/FantasyZoneTheMaze-Sms'

In [16]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path=models_dir)

In [17]:
env = KungFuKid()
env = Monitor(env, log_dir)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')

In [18]:
model_params = study.best_params

In [19]:
model = PPO('CnnPolicy', env, tensorboard_log=log_dir, verbose=1, **model_params)

Using cuda device
Wrapping the env in a VecTransposeImage.


c:\Users\rifat\miniconda3\envs\tf\lib\site-packages\stable_baselines3\ppo\ppo.py:140: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 13659`, after every 213 untruncated mini-batches, there will be a truncated mini-batch of size 27
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=13659 and n_envs=1)
  warnings.warn(


In [22]:
model.load('D:/torch_rl/AI_FOR_games/maze/models/FantasyZoneTheMaze-Sms/best_model_10010000.zip')

In [23]:
model.learn(total_timesteps=10000000, callback=callback)

Logging to logs/FantasyZoneTheMaze-Sms\PPO_7
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 2.89e+03 |
|    ep_rew_mean     | -5.12    |
| time/              |          |
|    fps             | 377      |
|    iterations      | 1        |
|    time_elapsed    | 36       |
|    total_timesteps | 13659    |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 2.97e+03   |
|    ep_rew_mean          | -6.16      |
| time/                   |            |
|    fps                  | 266        |
|    iterations           | 2          |
|    time_elapsed         | 102        |
|    total_timesteps      | 27318      |
| train/                  |            |
|    approx_kl            | 0.44516408 |
|    clip_fraction        | 0.444      |
|    clip_range           | 0.14       |
|    entropy_loss         | -2.71      |
|    explained_variance   | 0.61       |


KeyboardInterrupt: 

In [ ]:
env.close()

In [6]:
done=False
while not done:
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)
    if reward > 0:
        print(reward)
        
    env.render()

In [7]:
reward

-0.9

In [13]:
for i in range(100):
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)
    env.render()

In [14]:
info

{'lives': 5, 'score': 0}

In [8]:
info

{'lives': 0, 'score': 181}